In [2]:
import igraph as ig
import os

In [11]:
data_path = '../data/hashtags/edges/'
edge_files = [file for file in os.listdir(data_path) if file.endswith('.txt')]

In [12]:
def format_edgelist(edge_list):
    pass

In [36]:
output_string = ''
for i, edge_file in enumerate(edge_files):
    edge_file_path = os.path.join(data_path, edge_file)
    with open(edge_file_path, 'r') as f:
        edges = f.readlines()
    users = []
    for edge in edges:
        u, v = edge.strip().split(',')
        if 'None' in v:
            continue
        u_user = u.split('/')[-3]
        v_user = v.split('/')[-3]
        users.append((u_user, v_user))

    g = ig.Graph.TupleList(users, directed=True)

    output_string += f't # {i}\n'
    for v in g.vs:
        output_string += f'v {v.index} 0\n'
    for e in g.es:
        output_string += f'e {e.source} {e.target} 0\n'
    output_string += '\n'


In [30]:
with open('lol.txt', 'w') as f:
    f.write(output_string)